In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!source /content/drive/MyDrive/colab_env/bin/activate

import sys
sys.path.append("/content/drive/MyDrive/colab_env/lib/python3.10/site-packages")

### pip install transformers==4.30.2 -> runtime restart -> run all
transformers.__version__ = 4.35.2 in colab enviorment

In [ ]:
# !pip install transformers==4.30.2
# print(transformers.__version__)

In [3]:
import os
import glob
import emoji
import jieba
import spacy
import opencc
import string
import gensim
import requests
import transformers
import spacy_transformers

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from spacy.tokens import DocBin
from stopwordsiso import stopwords
from gensim.test.utils import datapath
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.pipeline.textcat_multilabel import Config, multi_label_cnn_config

## Word2Vec extract topic related keyword

In [5]:
os.chdir('/content/drive/MyDrive/Capstone')

In [ ]:
model = Word2Vec.load('./Spacy/word2vec/1209_w2v_try.bin')

## Labeled data w/ keyword

In [6]:
df_emoji = pd.read_csv('./Spacy/data/df_emoji.csv', encoding = 'utf-8')
df_remove_emoji = pd.read_csv('./Spacy/data/df_remove_emoji.csv', encoding = 'utf-8')
df_emoji_to_desc = pd.read_csv('./Spacy/data/df_emoji2desc.csv', encoding = 'utf-8')

In [ ]:
emoji_list = emoji.EMOJI_DATA

In [ ]:
def emoji2description_zh(text):
  return emoji.replace_emoji(text, replace = lambda chars, emoji_list: ' ' + ' '.join(emoji_list['zh'].split('_')).strip(':') + ' ')

### Emoji

In [ ]:
team_emoji = ['☀', '⚡', '🌚', '🌝', '🌞', '🐳', '🐋', '🍊', '🏎', '🔆', '🐯', '🦁', '🦅']
cheer_emoji = ['⛽', '🏐', '💓', '💕', '💖', '💗', '💘', '💙', '💚', '💛', '🏆', '👑', '🎉', '🎊', '❤', '🦾', '💯', '🙏', '💪', '🔥', '😍', '🤩', '🫶']

### Keyword

<h4> Player name

In [7]:
os.chdir('./team_player/')

In [8]:
file = glob.glob('./player/*.txt')

player_name = []
for t in file:
    with open(t, 'r') as f:
        names = [name.strip() for name in f.readlines()]
        player_name.extend(names)

<h4> Team

In [9]:
team_file = './team/team.txt'
team = []
with open(team_file, 'r') as f:
    team_name = [t.strip() for t in f.readlines()]

team_name.append('球隊')

<h4> Cheer / Tactic / Coach / Stream / Judge

In [10]:
cheer_words = ['加油', '恭喜', '最棒', '穩住', '撐住', '拼', '催', '軋油', '催蕊', '催落', '水', '一鼓作氣', '棒棒', '衝啊', '再接再厲', '追分', '沒事', '保持', '穩穩', '咬住', '穩贏', 'go', '無敵', '777', '666', '緊張', '爽',  '刺激', '喔喔', '精彩', '好球', '來了', 'OMG', '厲害', '很棒', '哈哈', 'ha', '激動', '狂', '神', '太強', '帥', '送啦', '好看', '讚', '好猛', '可惜', '耶', '鬼', 'lucky','wow', 'yey', 'yes','+u', '贏球', '水喔', '氣勢', '猛', '感動', '贏', '好棒', '強']
tactic_words = ['戰術', '攔網', '快攻', '進攻', '打點', '后排', '後排', '前排', '越網', '發球', '攻擊', '防守', '變陣', '接發', '救', '接球', '到位', '接發球', '發球', '攔', '舉球', '一傳', '角度', '主攻', '二傳', '輪轉', 'block', '封網', '失誤', '卡位', '觸球', '暫停', '換人', '落地', '得分', '舉', '局末', 'deuce', 'duce', 'ace', 'deu', '吊球', '短球', 'touch', '平手', '觸網', '自由', '連擊', '欄網', '欄中']
coach_words = ['連董','臉董', '小牛', '牛', '鄧公', '教練', '教练', '大康', '康胖', '舟', '連懂', '恩崇', '名鋒', '名峰', 'k總', '顯丞', '紹平', '邵平', '惠方', '教頭']
stream_words = ['導播', '小編', '球評', '賽評', '主播', '轉播', '講評', '聊天室', '畫面', '亞理', '亞里', '直播', '播報', '致平', '畫質', '邱老師', '電視', '邱老师', '000人', '000 人', '00', '千', 'hop sports', '鏡頭', '攝影機', 'hop', '看球', '人數', '視角', '重播', '運鏡', '鏡位', '延遲', '祐哥', '破萬', '萬人', '線上', '播報', '計分', '記分', '比數', '攝影', '主持', '評論員', '掌鏡', '收音', '致平', '嘉永', '美靜', '雜音', '訊號', '卡卡', '雜訊', '捷盛']
judge_words = ['裁判', '誤判', '挑戰', '規則', '主審', '吹', '鷹眼', '黃牌', '抗議', '判', '壓線', '紅牌', '主裁', '副裁','球証', '線審', '踩線', '採線', 'in', '界內', '持球', '犯規', '黑哨', '副審', '二審', '挑戰', '跳戰', '編審', '邊審', '一裁', '公正', '警告']

### Label data

In [11]:
## 加油
def contains_cheer_emoji(text): ## 4075
    return any(e in text for e in cheer_emoji)
def contains_cheer_word(text):  ## 31860
    return any(word in text for word in cheer_words)

## 轉播
def contains_stream_word(text):  ## 4156
    return any(word in text for word in stream_words)

## 教練
def contains_coach_word(text): ## 3384
    return any(word in text for word in coach_words)

## 戰術
def contains_tactic_word(text): ## 9758
    return any(word in text for word in tactic_words)

## 球員
def contains_player_name(text): ## 32041
    return any(word in text for word in player_name)

## 球隊
def contains_team_name(text):  ## 18897
    return any(word in text for word in team_name)
def contains_team_emoji(text): ## 5165
    return any(word in text for word in team_emoji)

## 裁判
def contains_judge_word(text):  ## 7266
    return any(word in text for word in judge_words)

In [12]:
len(df_emoji[df_emoji['text'].astype(str).apply(contains_player_name)])

32041

In [ ]:
no_condition_rows = df_emoji[
    (~df_emoji['text'].apply(contains_cheer_emoji)) &
    (~df_emoji['text'].apply(contains_cheer_word)) &
    (~df_emoji['text'].apply(contains_coach_word)) &
    (~df_emoji['text'].apply(contains_tactic_word)) &
    (~df_emoji['text'].apply(contains_team_emoji)) &
    (~df_emoji['text'].apply(contains_player_name)) &
    (~df_emoji['text'].apply(contains_team_name)) &
    (~df_emoji['text'].apply(contains_stream_word)) &
    (~df_emoji['text'].apply(contains_judge_word))
]

count_no_condition_rows = len(no_condition_rows)

print(f"Number of rows not meeting any condition: {count_no_condition_rows}")  ## 40381

Number of rows not meeting any condition: 40381


## Generate Training data

### 比賽 / 加油 / 轉播 / 球迷的聊天

In [ ]:
df_h1 = df_emoji.copy()
df_h1['text'] = df_h1['text'].astype(str)

In [ ]:
df_h1['比賽'] = ''
df_h1['加油'] = ''
df_h1['轉播'] = ''
df_h1['閒聊'] = ''

for i, row in df_h1.iterrows():

    if contains_cheer_emoji(row['text']) or contains_cheer_word(row['text']):
        df_h1.at[i, '加油'] = 1  ## 31284 -> 12/19 -> 34616
    else:
        df_h1.at[i, '加油'] = 0

    if contains_coach_word(row['text']) or contains_player_name(row['text']) or contains_tactic_word(row['text']) or contains_team_name(row['text']) or contains_team_emoji(row['text']) or contains_judge_word(row['text']):
        df_h1.at[i, '比賽'] = 1  ## 59056 -> 12/19 -> 63638
    else:
        df_h1.at[i, '比賽'] = 0

    if contains_stream_word(row['text']):
        df_h1.at[i, '轉播'] = 1  ## 4156
    else:
        df_h1.at[i, '轉播'] = 0

    if not (contains_cheer_emoji(row['text']) or contains_cheer_word(row['text']) or contains_coach_word(row['text']) or contains_player_name(row['text']) or contains_tactic_word(row['text']) or contains_team_name(row['text']) or contains_team_emoji(row['text']) or contains_judge_word(row['text']) or contains_stream_word(row['text'])):
        df_h1.at[i, '閒聊'] = 1  ## 46847 -> 12/19 -> 40381
    else:
        df_h1.at[i, '閒聊'] = 0

In [ ]:
len(df_h1[df_h1['閒聊'] == 1])

40381

In [ ]:
df_h1[df_h1['球迷的聊天'] == 1][9951:10000]

In [ ]:
os.chdir('/content/drive/MyDrive/Capstone/Spacy/data')

In [ ]:
df_h1.to_csv('Overall_Layer_topic.csv')

### 比賽 -> 球員 / 教練 / 裁判 / 戰術 / 球隊

In [ ]:
df_h2 = df_h1[df_h1['比賽'] == 1][['file', 'user', 'time', 'text']]
df_h2['text'] = df_h2['text'].astype(str)

In [ ]:
df_h2['球員'] = ''
df_h2['教練'] = ''
df_h2['裁判'] = ''
df_h2['戰術'] = ''
df_h2['球隊'] = ''

for i, row in df_h2.iterrows():

    if contains_player_name(row['text']):
        df_h2.at[i, '球員'] = 1
    else:
        df_h2.at[i, '球員'] = 0

    if contains_coach_word(row['text']) :
        df_h2.at[i, '教練'] = 1
    else:
        df_h2.at[i, '教練'] = 0

    if contains_judge_word(row['text']):
        df_h2.at[i, '裁判'] = 1
    else:
        df_h2.at[i, '裁判'] = 0

    if contains_tactic_word(row['text']):
        df_h2.at[i, '戰術'] = 1
    else:
        df_h2.at[i, '戰術'] = 0

    if contains_team_name(row['text']) or contains_team_emoji(row['text']):
        df_h2.at[i, '球隊'] = 1
    else:
        df_h2.at[i, '球隊'] = 0

In [ ]:
df_h2.to_csv('Game_Layer_topic.csv')

### 球迷的聊天

In [ ]:
os.chdir('/content/drive/MyDrive/Capstone/Spacy/data')

In [ ]:
df_h3 = df_h1[df_h1['閒聊'] == 1][['file', 'user', 'time', 'text']]

In [ ]:
df_h3.to_csv('Chat_Layer.csv')

In [ ]:
df_h3

,file,user,time,text
12,chat_1Xw3Kmbd_a8.json,Barry,8:35,哦哦原來 感謝解惑
14,chat_1Xw3Kmbd_a8.json,Ppppppj,11:39,打大專之後接企聯好累
16,chat_1Xw3Kmbd_a8.json,村田千夜,12:47,又被…
21,chat_1Xw3Kmbd_a8.json,QQKiu,16:57,2星期後才上場
23,chat_1Xw3Kmbd_a8.json,makubex7788,17:34,人員登錄 兩週才可能隨隊了 xdd
...,...,...,...,...
121284,chat_SAgpRa1oSUA.json,啟恆,2:38:57,恨鐵不成鋼的恨
121295,chat_SAgpRa1oSUA.json,村田千夜,2:41:00,笑得很開心
121298,chat_SAgpRa1oSUA.json,村田千夜,2:41:25,這場看到賺到
121304,chat_SAgpRa1oSUA.json,Slin,2:42:27,一定夯
